# Headway analysis and figure making

In [ ]:
import time as systime
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = 'Arial'
plt.style.use('default')
font = {'family' : 'Arial', 'size'   : 8}
plt.rc('font', **font)
plt.rcParams['mathtext.fontset'] = 'cm'
from tqdm import tqdm

import visual_utils as vis

figure_path = r'C:/SURFdrive/PhD progress/PhDResearch/3_AVCF/PartC/Figures/'

## 1 Datasets

In [ ]:
def read_data(cfpair, dataset):
    data = pd.read_hdf('../Data/InputData/'+dataset+'/CFdata/'+cfpair+'.h5', key='data').reset_index(drop=True)
    regimes = pd.read_hdf('../Data/OutputData/CF regime/'+dataset+'/regimes/regimes_all_'+cfpair+'.h5', key='regimes')
    data = data.merge(regimes, on=['case_id','time'], how='left')

    # correct negative speeds due to filtering
    data.loc[(data['v_follower']<0),'v_follower'] = 0.
    data.loc[(data['v_leader']<0),'v_leader'] = 0.

    data['dhw'] = data['x_leader'] - data['x_follower']
    data['thw'] = data['dhw']/data['v_follower']
    data.loc[np.isinf(data['thw']),'thw'] = np.nan
    
    return data   

Waymo_AH = read_data('AH', 'Waymo')
Waymo_HA = read_data('HA', 'Waymo')
Waymo_HH = read_data('HH', 'Waymo')
Lyft_AH = read_data('AH', 'Lyft')
Lyft_HA = read_data('HA', 'Lyft')
Lyft_HH = read_data('HH', 'Lyft')

### 1.1 Dataset selection

In [ ]:
print('Lyft:')
print('AH train min duration:', Lyft_AH.groupby('case_id').time.count().min()/10)
print('HA train min duration:', Lyft_HA.groupby('case_id').time.count().min()/10)
print('HH train min duration:', Lyft_HH.groupby('case_id').time.count().min()/10)
print('Waymo:')
print('AH train min duration:', Waymo_AH.groupby('case_id').time.count().min()/10)
print('HA train min duration:', Waymo_HA.groupby('case_id').time.count().min()/10)
print('HH train min duration:', Waymo_HH.groupby('case_id').time.count().min()/10)

In [ ]:
# Waymo
dataset_description = pd.DataFrame({'dataset':['Waymo_AH','Waymo_HA','Waymo_HH']})
dataset_description = dataset_description.set_index('dataset')
for data, setname in zip([Waymo_AH, Waymo_HA, Waymo_HH],['Waymo_AH','Waymo_HA','Waymo_HH']):
    test = data.groupby('case_id').agg({'v_follower':['min','max'],'v_leader':['min','max'], 'time':'max'})
    dataset_description.loc[setname,'num cases'] = len(test)
    # valid cases should have diverse car-following states
    valid_cases = test[(test['v_follower']['min']<0.1)&
                       (test['v_leader']['min']<0.1)&
                       (test['v_follower']['max']>=10)]
    dataset_description.loc[setname,'num valid cases'] = len(valid_cases)
    dataset_description.loc[setname,'average duration of valid cases'] = valid_cases['time']['max'].mean()

dataset_description.round(2)

In [ ]:
# Lyft
dataset_description = pd.DataFrame({'dataset':['Lyft_AH','Lyft_HA','Lyft_HH']})
dataset_description = dataset_description.set_index('dataset')
for data, setname in zip([Lyft_AH, Lyft_HA, Lyft_HH],['Lyft_AH','Lyft_HA','Lyft_HH']):
    test = data.groupby('case_id').agg({'v_follower':['min','max'],'v_leader':['min','max'], 'time':'max'})
    dataset_description.loc[setname,'num cases'] = len(test)
    # valid cases should have diverse car-following states
    valid_cases = test[(test['v_follower']['min']<0.1)&
                       (test['v_leader']['min']<0.1)&
                       (test['v_follower']['max']>=10)]
    dataset_description.loc[setname,'num valid cases'] = len(valid_cases)
    dataset_description.loc[setname,'average max. speed in valid cases'] = valid_cases['time']['max'].mean()

dataset_description.round(2)

### 1.2 Lyft overview

In [ ]:
fig, axes = vis.compare_HH_HA(Lyft_HH, Lyft_HA)

In [ ]:
fig.savefig(figure_path+'HH_HA_va.pdf', bbox_inches='tight', dpi=600)

### 1.3 Sensor error

#### 1.3.1 Length distribution
Unsure the influence of sensor error.

In [ ]:
fig, axes = vis.length_dist(Waymo_HH,
                            Waymo_HA, 'Waymo')

In [ ]:
fig.savefig(figure_path+'length_dist_waymo.pdf', bbox_inches='tight', dpi=600)

In [ ]:
fig, axes = vis.length_impact(Waymo_HH[(Waymo_HH['v_leader']<0.1)&(Waymo_HH['v_follower']<0.1)],
                              Waymo_HA[(Waymo_HA['v_leader']<0.1)&(Waymo_HA['v_follower']<0.1)])

In [ ]:
fig.savefig(figure_path+'length_impact_waymo.pdf', bbox_inches='tight', dpi=600)

In [ ]:
fig, axes = vis.length_dist(Lyft_HH,
                            Lyft_HA, 'Lyft')

In [ ]:
fig.savefig(figure_path+'length_dist_lyft.pdf', bbox_inches='tight', dpi=600)

In [ ]:
fig, axes = vis.length_impact(Lyft_HH[(Lyft_HH['v_leader']<0.1)&(Lyft_HH['v_follower']<0.1)],
                              Lyft_HA[(Lyft_HA['v_leader']<0.1)&(Lyft_HA['v_follower']<0.1)])

In [ ]:
fig.savefig(figure_path+'length_impact_lyft.pdf', bbox_inches='tight', dpi=600)

In [ ]:
cfdata_HH = Lyft_HH[(Lyft_HH['v_leader']<0.1)&(Lyft_HH['v_follower']<0.1)]
cfdata_HH = cfdata_HH.groupby('case_id')[['l_follower','l_leader']].first()
cfdata_HA = Lyft_HA[(Lyft_HA['v_leader']<0.1)&(Lyft_HA['v_follower']<0.1)]
cfdata_HA = cfdata_HA.groupby('case_id')[['l_follower','l_leader']].first()
length_HH = (cfdata_HH.l_follower + cfdata_HH.l_leader).mean()
length_HA = (cfdata_HA.l_follower + cfdata_HA.l_leader).mean()
print('Lyft impacts on gap', length_HA, length_HH, (length_HA-length_HH)/2)

cfdata_HH = Waymo_HH[(Waymo_HH['v_leader']<0.1)&(Waymo_HH['v_follower']<0.1)]
cfdata_HH = cfdata_HH.groupby('case_id')[['l_follower','l_leader']].first()
cfdata_HA = Waymo_HA[(Waymo_HA['v_leader']<0.1)&(Waymo_HA['v_follower']<0.1)]
cfdata_HA = cfdata_HA.groupby('case_id')[['l_follower','l_leader']].first()
length_HH = (cfdata_HH.l_follower + cfdata_HH.l_leader).mean()
length_HA = (cfdata_HA.l_follower + cfdata_HA.l_leader).mean()
print('Waymo impacts on gap', length_HA, length_HH, (length_HA-length_HH)/2)

### 1.4 Min. distance and time headway (baseline)

In [ ]:
fig, axes = vis.headway_dist(Waymo_HH,Waymo_HA)

In [ ]:
fig, axes = vis.headway_dist(Lyft_HH,Lyft_HA)

In [ ]:
fig.savefig(figure_path+'headway_dist_original.pdf', bbox_inches='tight', dpi=600)

## 2 CF regime

### 2.1 Regime distribution

In [ ]:
fig, axes = vis.regime_example(Lyft_HA)

In [ ]:
# fig.savefig(figure_path+'regime_example.pdf', bbox_inches='tight', dpi=600)

In [ ]:
regime_list_HA = pd.read_csv(r'U:/Vehicle Coordination Yiru/OutputData/CFAV/headway/CF regime/Lyft/regimes/regimes_list_HA.csv')
regime_list_HH = pd.read_csv(r'U:/Vehicle Coordination Yiru/OutputData/CFAV/headway/CF regime/Lyft/regimes/regimes_list_HH.csv')

In [ ]:
fig, ax = vis.regime_proportion(regime_list_HA, regime_list_HH)

In [ ]:
fig.savefig(figure_path+'regime_proportion.pdf', bbox_inches='tight', dpi=600)

In [ ]:
# thw HH
thw_HH = Lyft_HH.loc[Lyft_HH.groupby('case_id')['thw'].idxmin()].groupby('regime').agg({'time':'count','thw':'mean'})
thw_HH['time'] = thw_HH['time']/thw_HH['time'].sum()*100
thw_HH.loc[['Fa','C','Fd','D','A','F']].T.round(2)

In [ ]:
# thw HA
thw_HA = Lyft_HA.loc[Lyft_HA.groupby('case_id')['thw'].idxmin()].groupby('regime').agg({'time':'count','thw':'mean'})
thw_HA['time'] = thw_HA['time']/thw_HA['time'].sum()*100
thw_HA.loc[['Fa','C','Fd','D','A','F']].T.round(2)

In [ ]:
fig, axes = vis.headway_dist_regime(Lyft_HH, Lyft_HA)

In [ ]:
fig.savefig(figure_path+'headway_dist_regime.pdf', bbox_inches='tight', dpi=600)

## 3 Leading vehicle variability effect

In [ ]:
def read_data(cfpair,model,incul_data=False,data_path='../Data/OutputData/Variability/'):
    parameters = pd.read_csv(data_path+model+'/parameters_Lyft_'+cfpair+'.csv', index_col=0)
    parameters = parameters.dropna()

    loss = pd.read_csv(data_path+model+'/loss_Lyft_'+cfpair+'.csv', index_col=0)
    loss = loss.loc[parameters.index]

    if incul_data:
        data = pd.read_hdf(data_path+'cfdata_idm_Lyft_'+cfpair+'.h5', key='data')
        data['dhw'] = data['x_leader'] - data['x_follower'] - data['l_follower']/2 + data['l_leader']/2
        data['thw'] = data['dhw'] / data['v_follower']

    return (data, parameters, loss) if incul_data else (parameters, loss)

### 3.1 IDM parameter comparison

In [ ]:
data_HA, idm_HA, loss_HA = read_data('HA', 'idm', incul_data=True)
data_HH, idm_HH, loss_HH = read_data('HH', 'idm', incul_data=True)

In [ ]:
# only non-zero compared
loss_HH.dropna().agg(['count','mean','std']).round(2)

In [ ]:
# only non-zero compared
loss_HA.dropna().agg(['count','mean','std']).round(2)

In [ ]:
fig, axes = vis.idm_parameters(idm_HH,idm_HA)

In [ ]:
fig.savefig(figure_path+'idm_parameters.pdf', dpi=600, bbox_inches='tight')

### 3.2 Gipps parameter comparison

In [ ]:
data_HA, gipps_HA, loss_HA = read_data('HA', 'gipps', incul_data=True)
data_HH, gipps_HH, loss_HH = read_data('HH', 'gipps', incul_data=True)

In [ ]:
# only non-zero compared
loss_HH.dropna().agg(['count','mean','std']).round(2)

In [ ]:
# only non-zero compared
loss_HA.dropna().agg(['count','mean','std']).round(2)

In [ ]:
fig, axes = vis.gipps_parameters(gipps_HH,gipps_HA)

In [ ]:
fig.savefig(figure_path+'gipps_parameters.pdf', dpi=600, bbox_inches='tight')

### 3.2 Cross following

In [ ]:
def read_data_crossfollow(cfpair, model):
    data = pd.read_hdf('../Data/OutputData/Variability/crossfollow/'+model+'/crossfollow_Lyft_'+cfpair+'.h5', key='data')
    regimes = pd.read_hdf('../Data/OutputData/Variability/regime_simulated/'+model+'_simulated/regimes/regimes_all_'+cfpair+'.h5', key='regimes')
    data = data.merge(regimes, on=['case_id','time'], how='left')
    return data

#### 3.2.1 IDM simulated

In [ ]:
thwHH = [1,1,1,1]
thwHA = [-0.9,-0.7,-0.95,-0.9]
binHH = [0.1,0.1,0.1,0.1]
binHA = [0.15,0.15,0.15,0.15]
for count in [0,1,2,3]:
    fHH_lHHhigherVar = read_data_crossfollow('fHH_lHHhigherVar_'+str(count), 'idm')
    fHH_lHHlowerVar = read_data_crossfollow('fHH_lHHlowerVar_'+str(count), 'idm')
    fig, axes = vis.headway_leader_variability(fHH_lHHhigherVar, fHH_lHHlowerVar, thwHH[count], thwHA[count], binHH[count], binHA[count])
    fig.savefig(figure_path+'headway_leader_variability_'+str(count)+'.pdf', bbox_inches='tight', dpi=600)

#### 3.2.1 Gipps simulated

In [ ]:
thwHH = [0.9,0.9,0.9,1]
thwHA = [-0.9,-0.5,-0.85,-0.95]
binHH = [0.1,0.1,0.1,0.1]
binHA = [0.15,0.15,0.15,0.15]
for count in [0,1,2,3]:
    fHH_lHHhigherVar = read_data_crossfollow('fHH_lHHhigherVar_'+str(count), 'gipps')
    fHH_lHHlowerVar = read_data_crossfollow('fHH_lHHlowerVar_'+str(count), 'gipps')
    fig, axes = vis.headway_leader_variability(fHH_lHHhigherVar, fHH_lHHlowerVar, thwHH[count], thwHA[count], binHH[count], binHA[count])
    fig.savefig(figure_path+'gipps_headway_leader_variability_'+str(count)+'.pdf', bbox_inches='tight', dpi=600)

## 4 AV dynamics

traindata, HH case num: 15111 HA case num: 13813

traindata, HH subcase num: 30246 HA subcase num: 30257

valdata, HH case num: 15112 HA case num: 13814

valdata, HH subcase num: 30241 HA subcase num: 30199

In [ ]:
inputdata_path = r'U:/Vehicle Coordination Yiru/OutputData/CFAV/headway/AV leader/data/'
outputdata_path = r'U:/Vehicle Coordination Yiru/OutputData/CFAV/headway/AV leader/results_lstm/'

### 4.1 Classification evaluation

In [ ]:
confusion_matrices = pd.read_csv(outputdata_path+'confusion_matrices.csv')
loss_records = pd.read_csv(outputdata_path+'loss_records.csv')

In [ ]:
fig, axes = vis.evaluate_classifier(confusion_matrices[confusion_matrices['num_epoches']<205].copy(),loss_records)

In [ ]:
fig.savefig(figure_path+'classifier_evaluation.pdf',dpi=600, bbox_inches='tight')

In [ ]:
zero_tests = []
for epoch in [10,85,145,200]:
    zero_test = pd.read_csv(outputdata_path+'zero_test'+str(epoch)+'.csv')
    zero_tests.append(zero_test)

fig, axes = vis.scatter_zero_test(zero_tests, [10,85,145,200])

In [ ]:
fig.savefig(figure_path+'static_state_test.pdf',dpi=600, bbox_inches='tight')

### 4.2 Classification results

In [ ]:
# 1: AV leader, 0: HV leader
def read_classified_data(setname):
    data_HAandHH = pd.read_hdf(inputdata_path+'data_'+setname+'_HAandHH.h5', key='data')
    label_HAandHH = pd.read_hdf(inputdata_path+'label_'+setname+'_HAandHH.h5', key='label')
    pred_HAandHH = pd.read_csv(outputdata_path+'prediction_'+setname+'.csv')

    label_HAandHH['prediction'] = pred_HAandHH.set_index('new_id').loc[label_HAandHH['new_id'].values]['prediction'].values
    pred_HAandHH = data_HAandHH[['label','new_id','case_id','subcase','time']].copy()
    pred_HAandHH['pred'] = label_HAandHH.set_index(['new_id']).loc[data_HAandHH['new_id'].values]['prediction'].values

    data_HAandHH = data_HAandHH.drop_duplicates(subset=['case_id','time']).drop(columns=['subcase'])
    aggregation = pred_HAandHH.groupby(['label','case_id','time'])['pred'].mean().reset_index().set_index(['label','case_id','time'])
    data_HAandHH['pred_time'] = aggregation.reindex(pd.MultiIndex.from_frame(data_HAandHH[['label','case_id','time']])).values
    aggregation = pred_HAandHH.groupby(['label','case_id'])['pred'].mean().reset_index().set_index(['label','case_id'])
    data_HAandHH['pred_case'] = aggregation.reindex(pd.MultiIndex.from_frame(data_HAandHH[['label','case_id']])).values

    HA = data_HAandHH[(data_HAandHH['label']>0.5)].copy()
    HA[['v_leader','a_leader','v_follower','dhw','thw']] = Lyft_HA.set_index(['case_id','time']).reindex(pd.MultiIndex.from_frame(HA[['case_id','time']]))[['v_leader','a_leader','v_follower','dhw','thw']].values
    HA['regime'] = Lyft_HA.set_index(['case_id','time']).reindex(pd.MultiIndex.from_frame(HA[['case_id','time']]))['regime'].values
    HH = data_HAandHH[(data_HAandHH['label']<0.5)].copy()
    HH[['v_leader','a_leader','v_follower','dhw','thw']] = Lyft_HH.set_index(['case_id','time']).reindex(pd.MultiIndex.from_frame(HH[['case_id','time']]))[['v_leader','a_leader','v_follower','dhw','thw']].values
    HH['regime'] = Lyft_HH.set_index(['case_id','time']).reindex(pd.MultiIndex.from_frame(HH[['case_id','time']]))['regime'].values

    return HA, HH

In [ ]:
# train_HA, train_HH = read_classified_data('train')
val_HA, val_HH = read_classified_data('val')

In [ ]:
print('In validation set:')
print('Num of HA leaders classified as HV', val_HA[val_HA['pred_case']<0.5]['case_id'].nunique())
print('Num of HA leaders classified as AV', val_HA[val_HA['pred_case']>0.5]['case_id'].nunique())
print('Num of HH leaders classified as HV', val_HH[val_HH['pred_case']<0.5]['case_id'].nunique())
print('Num of HH leaders classified as AV', val_HH[val_HH['pred_case']>0.5]['case_id'].nunique())

In [ ]:
fig, axes = vis.compare_leader(val_HA[val_HA['pred_case']<=0.5], val_HA[val_HA['pred_case']>0.5], Lyft_HH)

In [ ]:
fig.savefig(figure_path+'compare_leader.pdf',dpi=600, bbox_inches='tight')

In [ ]:
fig, axes = vis.headway_dist_dynamics(val_HA, 'pred_case')

In [ ]:
fig.savefig(figure_path+'headway_dist_dynamics_case.pdf', dpi=600, bbox_inches='tight')

In [ ]:
fig, axes = vis.headway_dist_dynamics(val_HA, 'pred_time')

In [ ]:
fig.savefig(figure_path+'headway_dist_dynamics_time.pdf', dpi=600, bbox_inches='tight')